<a href="https://colab.research.google.com/github/Rehabibraheem/datasciencecoursera/blob/master/rehab_(2_classes)_EEG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import librosa


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Egg_file=loadmat('/content/drive/MyDrive/Rehab_Egg_code/BCICIV_calib_ds1d.mat')
Egg_file=loadmat('/content/drive/MyDrive/Colab Notebooks/project/BCICIV_1_mat.zip (Unzipped Files)/BCICIV_calib_ds1e.mat')

In [6]:
data=Egg_file['cnt']
Pos=Egg_file['mrk']['pos'][:,0].item().ravel()
Tag=Egg_file['mrk']['y'][:,0].item().ravel()
fs=Egg_file['nfo']['fs'][0,0].item()

## Construct signal Frames

In [7]:
from sklearn import preprocessing
#import emd
#N_Ch=5
Tn=Tag.shape[0]
Frames=np.ones((Tn,250))
c=0
shift=40
for t in range(0,200):
    #for c in range(0,N_Ch):
        s0 = data[Pos[t]+shift:Pos[t] + 250+shift,0]
        s1 = data[Pos[t]+shift:Pos[t] + 250+shift,1]
        s2 = data[Pos[t]+shift:Pos[t] + 250+shift,2]
        s=s0+s1+s2
        x=s #(s*np.hamming(nfft))
        nx=(x - np.mean(x)) / np.max(np.abs(x - np.mean(x)))
        Frames[t,:] =nx

    #s = xx  # Assuming signal and winfunc are defined elsewhere for simplicity
    #sw = s * winfunc
    #x = sw
    #y[t, 0] = Dn[t]
    #ss,_=librosa.effects.trim(nx,top_db=190)


In [8]:
del s,x,nx,c,t,shift

In [9]:
N_mel=40
mfcc_xc=librosa.feature.mfcc(y=Frames, sr=fs,n_mfcc=N_mel,n_fft=64,hop_length=32,htk=True,fmin=5,fmax=30,dct_type=2)
ID_mfcc = mfcc_xc.reshape((Tn), -1)

/usr/local/lib/python3.10/dist-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


##Gapor Filter

In [10]:
from scipy.fft import fft
LowF = 5  # Example value for LowF
HiF = 30  # Example value for HiF
Fs = 100  # Example value for Fs
Chs = 50  # Example value for Ch
R=50
H_Norm=np.zeros((+R*2,Chs),dtype=complex)
Fc = librosa.mel_frequencies(n_mels=Chs, fmin=LowF, fmax=HiF, htk=False)
Fcq=librosa.cqt_frequencies(n_bins=Chs, fmin=LowF, bins_per_octave=8, tuning=6.6)
#freq_edges, freq_centres = emd.spectra.define_hist_bins(50, 3000, Ch, 'log')

Lfcenters =Fs* (np.linspace(LowF/Fs, HiF/Fs, Chs))
#CBw =25 + 75 * (1 + 1.25 * (Lfcenters / 1000) ** 2) ** 0.69
CBw =np.ones(Chs)*2.0
for m in range(0,Chs):
    fc=Lfcenters[m]
    alpha = np.sqrt(2*np.pi) * CBw[m] / Fs
    Oc = 2 * np.pi * fc / Fs
    indices = np.arange(-R + 1, R + 1)  # Generate array indices [-N+1:N]
    g = np.exp(-1 * (alpha * indices) ** 2)
    indices2 = np.arange(0, 2*R-0)  # Generate array indices [0:2*N-1]
    c = np.cos(Oc * indices2)
    h=g*c
    H = fft(h.T)
    Norm=np.max(H)
    H_Norm[:,m] = H/Norm

In [ ]:
del   Fcq,Fc,CBw,LowF, HiF,Fs,R, Lfcenters, fc, alpha, Oc, indices, g, indices2, c, h, H, Norm, m

In [ ]:
del X_test, X_train, y_pred, y_test,y_train

In [ ]:
del s

##WIF2011

In [14]:
from scipy.fft import ifft
from scipy.signal import hilbert
i=4
nfft=H_Norm.shape[0]
Test = np.ones((i,Chs))
wIF = np.ones((Tn,i,Chs))
fs=100

for t in range(0,Tn):
    x=Frames[t,:]
    step=0
    for j in range(0,i):
        xc=x[step:step+100]
        #w=(xc*np.hamming(nfft))
        X=fft(xc,n=nfft)
        for c in range(0,Chs):
            Xf=(X)*(H_Norm[:,c])
            xf=np.real(ifft(Xf))
            #En=(np.abs((X)@(H_Norm[:,c])))
            ht =hilbert(xf)
            real_part = np.real(ht)
            imag_part = np.imag(ht)
            iamp = np.sqrt(real_part**2 + imag_part**2)
            phase = np.unwrap(np.angle(ht))
            diff_phase = np.diff(phase)
            ifreq = fs * diff_phase / (2 * np.pi)
            IF=(np.sum(ifreq * iamp[:-1]**2)) / (np.sum(iamp[:-1]**2))
            #eIF=complex(IF,En)
            Test[j,c] =IF
            wIF[t,j,c] = IF
            #wIF[t,j,c] =np.real(eIF)
        step=step+50

In [15]:
del nfft,t,x,step,j,xc,X,c,Xf,xf,ht,real_part,imag_part,phase,iamp,diff_phase,ifreq,IF,Test

In [ ]:
plt.plot(Test)

In [17]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
colors = ['purple','red','tan', 'blue', 'brown', 'green', 'pink', 'cyan', 'navy', 'orange', 'yellow','black']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

In [ ]:
Y=Tag

In [18]:
ID = wIF.reshape(200, -1)
#ID=ID_IF2
#ID=wIF2088

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=1)
X_r = lda.fit(ID, Y).transform(ID)

In [20]:
plt.scatter(X_r[:,0],X_r[:,0],c=vectorizer(Y.astype(int)))


NameError: name 'X_r' is not defined

In [21]:
plt.scatter(X_r[:,0],X_r[:,0],c=vectorizer(Y.astype(int)))


NameError: name 'X_r' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_r, Y,test_size=0.5)

#lda = LinearDiscriminantAnalysis(n_components=1)
lda.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = lda.predict(X_test)
print(accuracy_score(y_test, y_pred))